Cell 1 — Mount Drive

In [1]:
# 1) Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Cell 2 — Configure what to create

In [2]:
# 2) What to create (you can make one or both)
PROJECT_DIR = "/content/drive/MyDrive/WorstellWitnessWaybackMachine"   # <- change if different

CREATE_FAMILY = True
CREATE_PLACE  = True

# ----- Family page settings -----
FAM_SLUG   = "volckertszen-dircks.md"   # file in docs/families/
FAM_TITLE  = "Volckertszen / Dircks (New Amsterdam ↔ Bushwick)"
FAM_SUM    = "A ship’s-carpenter family at Smit’s Vly paired with a shore plot at Noorman’s Kill/Bushwick."

# ----- Place page settings -----
PLACE_SLUG  = "normans-kill-bushwick.md"  # file in docs/places/
PLACE_TITLE = "Noorman’s (Norman’s) Kill — Bushwick/Greenpoint Shore, Long Island"
PLACE_SUM   = "Opposite Smit’s Vly: meadows, inlets, and small farms; ferries and skiffs cross daily."

# Optional: link the pages to each other
LINK_TO_THIS_FAMILY = f"../families/{FAM_SLUG}"
LINK_TO_THIS_PLACE  = f"../places/{PLACE_SLUG}"

print("Project:", PROJECT_DIR)
print("Family file:", FAM_SLUG)
print("Place file :", PLACE_SLUG)

Project: /content/drive/MyDrive/WorstellWitnessWaybackMachine
Family file: volckertszen-dircks.md
Place file : normans-kill-bushwick.md


Cell 3 — Ensure folders and write templates

In [3]:
# 3) Write family & place pages from templates (idempotent; will overwrite if you choose)

from pathlib import Path
import datetime, textwrap

docs = Path(PROJECT_DIR, "docs")
families_dir = docs / "families"
places_dir   = docs / "places"
families_dir.mkdir(parents=True, exist_ok=True)
places_dir.mkdir(parents=True, exist_ok=True)

fam_path   = families_dir / FAM_SLUG
place_path = places_dir / PLACE_SLUG

timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

family_md = f"""---
title: {FAM_TITLE}
summary: {FAM_SUM}
---

**Headline:** *Dirck Volckertszen* — called **“de Noorman”** in New Amsterdam papers — keeps house near **Smit’s Vly** with **Christina (Vigne)**; their son **Volkert Dircks** is baptized **1643** (DRC).

!!! info "Crowd feel (1640s–1650s)"
    - Smit’s Vly lanes: few dozen households; ship carpenters, ferrymen, coopers
    - Bushwick shore: scattered farms, hay meadows; ferries & skiffs cross daily
    - Languages: Dutch, Low German, French, Scandinavian, Lenape; some English

## People & household
- **Dirck Volckertszen (“de Noorman”)** — ship’s carpenter / boatwright.
- **Christina (Vigne)** — East River family network.
- **Volkert Dircks** — bapt. 1643 (DRC New Amsterdam).

## Where to look for them
- DRC registers; court/notarial minutes; Castello Plan; Bushwick/Long Island town papers.

## Linked pages
- Place: [Noorman’s Kill / Bushwick shore]({LINK_TO_THIS_PLACE})

*Scaffold created {timestamp}. Add citations as you collect folio/entry lines.*
"""

place_md = f"""---
title: {PLACE_TITLE}
summary: {PLACE_SUM}
---

A **low, watery fringe** of creeks and inlets along the Long Island side of the East River (later **Bushwick Inlet / Greenpoint**). Skiffs cross for hay, timber, garden produce, and people.

!!! info "Crowd feel (mid-17th century)"
    - Households: dozens by mid-century, scattered along lanes and meadows
    - Traffic: ferries and workboats; carts, cattle, hay loads
    - Neighbors: Breukelen, Amersfoort; Newtown Creek

## Why it matters here
- Pairs with **Smit’s Vly** on Manhattan: work, worship, court, and market across the water.
- A realistic **“home + shore plot”** pattern for a ship-carpenter family.

## Linked pages
- Family: [Volckertszen / Dircks]({LINK_TO_THIS_FAMILY})
- Nearby: [Smit’s Vly](../places/smits-vly.md)

*Scaffold created {timestamp}. Add maps, parcels, and citations as you confirm.*
"""

if CREATE_FAMILY:
    fam_path.write_text(family_md, encoding="utf-8")
    print("Wrote family page:", fam_path)

if CREATE_PLACE:
    place_path.write_text(place_md, encoding="utf-8")
    print("Wrote place page :", place_path)

Wrote family page: /content/drive/MyDrive/WorstellWitnessWaybackMachine/docs/families/volckertszen-dircks.md
Wrote place page : /content/drive/MyDrive/WorstellWitnessWaybackMachine/docs/places/normans-kill-bushwick.md


Cell 4 — Update mkdocs.yml navigation (adds sections if missing)

In [4]:
# 4) Update mkdocs.yml nav safely (adds sections if needed)

import re
from pathlib import Path

cfg_path = Path(PROJECT_DIR, "mkdocs.yml")
cfg = cfg_path.read_text(encoding="utf-8")

def ensure_section(cfg, section_label):
    if re.search(rf"^  - {re.escape(section_label)}:\s*$", cfg, flags=re.M):
        return cfg  # already present
    # insert after 'nav:' block start
    if "nav:\n" in cfg:
        return re.sub(r"(nav:\n)", rf"\1  - {section_label}:\n", cfg, count=1)
    # if somehow no nav: create a minimal one
    return "nav:\n  - Home: index.md\n  - " + section_label + ":\n" + cfg

cfg = ensure_section(cfg, "Families")
cfg = ensure_section(cfg, "Places")

def add_nav_line(cfg, section_label, title, relpath):
    line = f"      - {title}: {relpath}\n"
    if relpath in cfg:
        return cfg  # already listed
    return re.sub(
        rf"(^  - {re.escape(section_label)}:\s*$)",
        rf"\1\n{line}",
        cfg,
        count=1, flags=re.M
    )

if CREATE_FAMILY:
    cfg = add_nav_line(cfg, "Families", FAM_TITLE, f"families/{FAM_SLUG}")
if CREATE_PLACE:
    cfg = add_nav_line(cfg, "Places",  PLACE_TITLE, f"places/{PLACE_SLUG}")

cfg_path.write_text(cfg, encoding="utf-8")
print("Navigation updated in:", cfg_path)

Navigation updated in: /content/drive/MyDrive/WorstellWitnessWaybackMachine/mkdocs.yml


Cell 5 — Build & preview the site locally

In [5]:
# 5) Build & preview
!python -m pip install -q mkdocs mkdocs-material
%cd $PROJECT_DIR
!mkdocs build -q

from IPython.display import HTML
HTML(open("site/index.html","r",encoding="utf-8").read())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.9/397.9 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 13.4 MB/s eta 0:00:00
/content/drive/MyDrive/WorstellWitnessWaybackMachine
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


(Optional) Cell 6 — Commit & push (works if your remote includes your token)

In [6]:
# 6) Optional: push to GitHub (requires tokenized 'origin' or saved credentials)
%cd $PROJECT_DIR
!git add -A
!git commit -m "Add family/place scaffolds: {FAM_SLUG}, {PLACE_SLUG}" || true
!git pull --rebase origin main || true
!git push -u origin main

/content/drive/MyDrive/WorstellWitnessWaybackMachine
[main 5cb6e9c] Add family/place scaffolds: volckertszen-dircks.md, normans-kill-bushwick.md
 3 files changed, 47 insertions(+), 83 deletions(-)
 rewrite docs/families/volckertszen-dircks.md (94%)
 rewrite docs/places/normans-kill-bushwick.md (93%)
From https://github.com/BillWorstell/WorstellWitnessWaybackMachine
 * branch            main       -> FETCH_HEAD
Current branch main is up to date.
Enumerating objects: 15, done.
Counting objects: 100% (15/15), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 2.06 KiB | 140.00 KiB/s, done.
Total 8 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/BillWorstell/WorstellWitnessWaybackMachine.git
   d02de76..5cb6e9c  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


Notes / tips

You can set CREATE_FAMILY or CREATE_PLACE to False to create just one page.

For internal links in MkDocs, relative paths like ../places/smits-vly.md are reliable.

If you ever want the remote cleaned back to a non-token URL, run (in a cell):
!git remote set-url origin https://github.com/BillWorstell/WorstellWitnessWaybackMachine.git